In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sys
from astropy import units as u 
import gc

from snewpy.neutrino import Flavor, MassHierarchy, MixingParameters
from snewpy.models.ccsn import Zha_2021, Nakazato_2013
from snewpy.flavor_transformation import AdiabaticMSW, NonAdiabaticMSWH, \
                                         TwoFlavorDecoherence, ThreeFlavorDecoherence, \
                                         NeutrinoDecay, AdiabaticMSWes, NonAdiabaticMSWes

mpl.rc('font', size=18)
%matplotlib inline

In [2]:
model = Nakazato_2013(progenitor_mass=20*u.solMass, revival_time=100*u.ms, metallicity=0.004, eos='shen')
#model = Nakazato_2013(progenitor_mass=20*u.solMass, revival_time=100*u.ms, metallicity=0.004, eos='shen')
model.get_time()

<Quantity [-5.00e-02, -4.50e-02, -4.00e-02, -3.50e-02, -3.00e-02,
           -2.50e-02, -2.00e-02, -1.50e-02, -1.00e-02, -5.00e-03,
            0.00e+00,  1.00e-03,  2.00e-03,  3.00e-03,  4.00e-03,
            5.00e-03,  6.00e-03,  7.00e-03,  8.00e-03,  9.00e-03,
            1.00e-02,  1.10e-02,  1.20e-02,  1.30e-02,  1.40e-02,
            1.50e-02,  1.60e-02,  1.70e-02,  1.80e-02,  1.90e-02,
            2.00e-02,  2.10e-02,  2.20e-02,  2.30e-02,  2.40e-02,
            2.50e-02,  2.60e-02,  2.70e-02,  2.80e-02,  2.90e-02,
            3.00e-02,  3.10e-02,  3.20e-02,  3.30e-02,  3.40e-02,
            3.50e-02,  3.60e-02,  3.70e-02,  3.80e-02,  3.90e-02,
            4.00e-02,  4.10e-02,  4.20e-02,  4.30e-02,  4.40e-02,
            4.50e-02,  4.60e-02,  4.70e-02,  4.80e-02,  4.90e-02,
            5.00e-02,  5.05e-02,  5.10e-02,  5.15e-02,  5.20e-02,
            5.25e-02,  5.30e-02,  5.35e-02,  5.40e-02,  5.45e-02,
            5.50e-02,  5.55e-02,  5.60e-02,  5.65e-02,  5.70e-02,
          

In [3]:
times= np.linspace(0,0.5,201)  << u.s
for i,t in enumerate(times):
    print(np.isscalar(times.value))

energies = np.linspace(1,51,101) * u.MeV    
d = (10*u.kpc).to('cm').value # distance to SN
        
t= 0.05  << u.s

ospec_nmo_tof = model.get_tof_transformed_spectra(t, energies,  AdiabaticMSW())
print(ospec_nmo_tof)

    

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [4]:
def plot_total_flux(model, xform_nmo, xform_imo):
    """Plot initial and oscillated neutrino luminosities.
    
    Parameters
    ----------
    model : SupernovaModel
        An input model from a CCSN simulation.
    flav_xform : FlavorTransformation
        A FlavorTransformation subclass; used to create an instance.
    """
    
    energies = np.linspace(1,51,101) * u.MeV    
    d = (10*u.kpc).to('cm').value # distance to SN
        
    times= np.linspace(0,0.2,101)  << u.s
    burst_epoch = times <= 0.3*u.s# 
    accretion_epoch = (times > 0.1*u.s) & (times <= 0.5*u.s)
    
    
    
    ilum = {}
    olum_nmo = {}
    olum_nmo_tof = {}
    
    for flavor in Flavor:#(0123)
        ilum[flavor] = np.zeros(len(times))
        olum_nmo[flavor] = np.zeros(len(times))
        olum_nmo_tof[flavor] = np.zeros(len(times))

    # Compute the transformed and untransformed flux at each time.
    for i, t in enumerate(times):
        #ispec = model.get_initial_spectra(t, energies)
        ospec_nmo = model.get_transformed_spectra(t, energies, xform_nmo)
        ospec_nmo_tof = model.get_tof_transformed_spectra(t, energies, xform_nmo)

        for flavor in Flavor:
            for j, E in enumerate(energies):
                #ispec[flavor][j] /= (4.*np.pi*d**2)
                ospec_nmo[flavor][j] /= (4.*np.pi*d**2)
                ospec_nmo_tof[flavor][0][j] /= (4.*np.pi*d**2)#列是色，行是能量，通量表面积
        
        for flavor in Flavor:
            #ilum[flavor][i] = np.trapz(ispec[flavor].to('1/(erg*s)'), energies.to('erg')).value #执行定积分，value提取数值不包含单位的部分
            
            olum_nmo[flavor][i] = np.trapezoid(ospec_nmo[flavor].to('1/(erg*s)'), energies.to('erg')).value
            olum_nmo_tof[flavor][i] = np.trapezoid(ospec_nmo_tof[flavor].to('1/(erg*s)'), energies.to('erg')).value            
            
    # make the figures 
    fig, ax = plt.subplots(1,1, figsize=(10,10), tight_layout=True)
    
    smax = [0.]
    titles = ['Nakazato_2013_total_flux_0.6eV']
    
    for j, phase in enumerate([burst_epoch]):
        
        timeunits =  's'
        for i, flavor in enumerate(Flavor):
            if i == 0:
                smax[j] = np.maximum(smax[j], 1.1*np.max(olum_nmo[1][phase]))
                    
            ax.plot(times[phase].to(timeunits),
                    olum_nmo[flavor][phase], label=f"{flavor.to_tex()} ", lw=3,
                    color='C0' if flavor==0 else 'C1' if flavor==1 else 'C2' if flavor==2 else 'C3',
                    ls=':' )
            ax.plot(times[phase].to(timeunits),
                    olum_nmo_tof[flavor][phase], label=f"{flavor.to_tex()} (tof_Transformed)", lw=3,
                    color='C0' if flavor==0 else 'C1' if flavor==1 else 'C2' if flavor==2 else 'C3',
                    ls= '-')
            
            
        ax.set(xlim=(times[phase][0].to(timeunits).value, times[phase][-1].to(timeunits).value),ylim=(0, smax[j]))
            
            
        ax.set(ylabel=r'flux [ erg$^{-1}$ cm$^{-2}$ s$^{-1}$]')
        ax.legend(loc='upper right', ncol=1, fontsize=18)
            
        ax.set(title=titles)
        
      
            
        ax.set(xlabel='time [{}]'.format(timeunits))
            
        ax.grid(ls=':')

      
    return fig


In [ ]:
fig = plot_total_flux(model, AdiabaticMSW(), AdiabaticMSW(mh=MassHierarchy.INVERTED) )

C:\Users\M_bum\AppData\Local\Temp\ipykernel_46572\676283548.py:46: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  olum_nmo_tof[flavor][i] = np.trapezoid(ospec_nmo_tof[flavor].to('1/(erg*s)'), energies.to('erg')).value


In [ ]:
def plot_total_flux(model, xform_nmo, xform_imo):
    """Plot initial and oscillated neutrino luminosities.
    
    Parameters
    ----------
    model : SupernovaModel
        An input model from a CCSN simulation.
    flav_xform : FlavorTransformation
        A FlavorTransformation subclass; used to create an instance.
    """
    
    energies = np.linspace(1,50,100) * u.MeV    
    d = (10*u.kpc).to('cm').value # distance to SN
    D=10* u.kpc
    mass=0.5* u.eV    
    times = model.get_time()
    burst_epoch = times <= 0.1*u.s
    accretion_epoch = (times > 0.1*u.s) & (times <= 0.5*u.s)
    cooling_epoch = (times > 0.5*u.s) & (times <= 10*u.s)
    
    ilum = {}
    olum_nmo = {}
    olum_imo = {}
    olum_nmo_tof = {}
    olum_imo_tof = {}
    
    for flavor in Flavor:
        ilum[flavor] = np.zeros(len(times))
        olum_nmo[flavor] = np.zeros(len(times))
        olum_imo[flavor] = np.zeros(len(times))
        olum_nmo_tof[flavor] = np.zeros(len(times))
        olum_imo_tof[flavor] = np.zeros(len(times))

    # Compute the transformed and untransformed flux at each time.
    for i, t in enumerate(times):
        ospec_nmo = model.get_transformed_spectra(t, energies, xform_nmo)
        ospec_imo = model.get_transformed_spectra(t, energies, xform_imo)
        ospec_nmo_tof = model.get_delayed_flux(D, t, energies, mass, AdiabaticMSW())
        ospec_imo_tof = model.get_delayed_flux(D, t, energies, mass, xform_imo)

        for flavor in Flavor:
            for j, E in enumerate(energies):
                ospec_nmo[flavor][j] /= (4.*np.pi*d**2)
                ospec_imo[flavor][j] /= (4.*np.pi*d**2)
                ospec_nmo_tof[flavor][j] /= (4.*np.pi*d**2)
                ospec_nmo_tof[flavor][j] /= (4.*np.pi*d**2)
        
        for flavor in Flavor: 
            olum_nmo[flavor][i] = np.trapz(ospec_nmo[flavor].to('1/(erg*s)'), energies.to('erg')).value
            olum_imo[flavor][i] = np.trapz(ospec_imo[flavor].to('1/(erg*s)'), energies.to('erg')).value
            olum_nmo_tof[flavor][i] = np.trapz(ospec_imo[flavor].to('1/(erg*s)'), energies.to('erg')).value
            olum_imo_tof[flavor][i] = np.trapz(ospec_imo[flavor].to('1/(erg*s)'), energies.to('erg')).value
            
    # make the figures 
    fig, axes = plt.subplots(2,3, figsize=(20,12), tight_layout=True)
    
    smax = [0.,0.,0.]
    titles = ['Untransformed', 'Transformed (NMO)', 'Transformed (IMO)']
    for i, (spec_olum, spec_ospec) in enumerate([(olum_nmo,olum_nmo_tof),(olum_imo,olum_imo_tof) ]):
        for j, phase in enumerate([burst_epoch, accretion_epoch, cooling_epoch]):
            ax = axes[i,j]
            timeunits = 'ms' if j==0 else 's'
                
            if i == 0:
                smax[j] = np.maximum(smax[j], 1.1*np.max(spec_olum[0][phase]))
                    
            ax.plot(times[phase].to(timeunits),
                    spec_olum[0][phase], label=f"{"NU_E"}", lw=3,
                    color='C0' ,
                    ls='-' )
            ax.plot(times[phase].to(timeunits),
                    spec_ospec[0][phase], label=f"{"NU_E"} ", lw=3,
                    color= 'C1',
                    ls= ':')
                
            
            ax.set(xlim=(times[phase][0].to(timeunits).value, times[phase][-1].to(timeunits).value),
                   ylim=(0, smax[j]))
            
            if j==0:
               ax.set(ylabel=r'flux [$10^{16}$ erg$^{-1}$ cm$^{-2}$ s$^{-1}$]')
               ax.legend(loc='upper right', ncol=1, fontsize=18)
            if j==1:
                ax.set(title=titles[i])
            if i < 2:
                ax.set(xticklabels=[])
            else:
                ax.set(xlabel='time [{}]'.format(timeunits))
            
            ax.grid(ls=':')

      
    return fig
fig = plot_total_flux(model, AdiabaticMSW(), AdiabaticMSW(mh=MassHierarchy.INVERTED) )

In [ ]:
    sys.setrecursionlimit(3000)
    energies = np.linspace(1,50,100) * u.MeV
    D=10* u.kpc
    d = (10*u.kpc).to('cm').value # distance to SN
    mass=0.5* u.eV    
    times = model.get_time()
    burst_epoch = times <= 0.1*u.s
    accretion_epoch = (times > 0.1*u.s) & (times <= 0.5*u.s)
    cooling_epoch = (times > 0.5*u.s) & (times <= 10*u.s)
    
    
    olum_nmo = np.zeros(len(times))
    #olum_imo = {E: np.zeros(len(times)) for E in energy_ranges}
    olum_nmo_tof = np.zeros(len(times))
    #olum_imo_tof = {E: np.zeros(len(times)) for E in energy_ranges}

    # Compute the transformed and untransformed flux at each time.
    for i, t in enumerate(times):
        #ispec = model.get_initial_spectra(t, energies)
        ospec_nmo = model.get_transformed_spectra(t, energies, AdiabaticMSW())
        #ospec_imo = model.get_transformed_spectra(t, energies, xform_imo)
        ospec_nmo_tof = model.get_delayed_flux(D, t, energies, mass, AdiabaticMSW())
        ospec_imo_tof,tof1 = model.get_delayed_flux(D, t, energies, mass, xform_imo)
        for flavor in Flavor:
            for j, E in enumerate(energies):
                ospec_nmo[flavor][j] /= (4. * np.pi * d ** 2)
                ospec_nmo_tof[flavor][j] /= (4. * np.pi * d ** 2)

            # 创建一个掩码以选择当前范围内的能量
        olum_nmo[i] = np.trapz(ospec_nmo[0].to('1/(erg*s)'), energies.to('erg')).value
                
            #olum_imo[energy_range][i] = np.trapezoid(ospec_imo[0][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
            
            #olum_nmo_tof[energy_range][i] = np.trapz(ospec_nmo_tof[0][mask].to('1/(erg*s)'), energies[mask].to('erg')).value
              
            #olum_imo_tof[energy_range][i] = np.trapezoid(ospec_imo_tof[0][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
            
    print(olum_nmo[0])

In [ ]:
def plot_total_flux(model, xform_nmo, xform_imo):
    """Plot initial and oscillated neutrino luminosities.
    
    Parameters
    ----------
    model : SupernovaModel
        An input model from a CCSN simulation.
    flav_xform : FlavorTransformation
        A FlavorTransformation subclass; used to create an instance.
    """
    T = np.linspace(1,101,201) * u.MeV 
    energy = np.arange(1, 50, 0.2) << u.MeV 
    D=10* u.kpc
    d = (10*u.kpc).to('cm').value # distance to SN
    mass=0.5* u.eV    
   
    
    
    #ilum = {E: {flavor: np.zeros(len(times)) for flavor in Flavor} for E in energy_ranges}
    # 初始化字典以存储每个能量范围和味道的未转换光度
    olum_nmo = {E: {flavor: np.zeros(len(times)) for flavor in Flavor} for E in energy_ranges}
    olum_imo = {E: {flavor: np.zeros(len(times)) for flavor in Flavor} for E in energy_ranges}
    olum_nmo_tof = {E: {flavor: np.zeros(len(times)) for flavor in Flavor} for E in energy_ranges}
    olum_imo_tof = {E: {flavor: np.zeros(len(times)) for flavor in Flavor} for E in energy_ranges}

    # Compute the transformed and untransformed flux at each time.
    for i, t in enumerate(times):
        #ispec = model.get_initial_spectra(t, energies)
        ospec_nmo = model.get_transformed_spectra(t, energies, xform_nmo)
        ospec_imo = model.get_transformed_spectra(t, energies, xform_imo)
        ospec_nmo_tof,tof1 = model.get_delayed_flux(D, t, energies, mass, xform_nmo)
        ospec_imo_tof,tof1 = model.get_delayed_flux(D, t, energies, mass, xform_imo)
        del tof1
        for energy_range in energy_ranges:
            # 遍历每个能量范围
            e_min, e_max = energy_range
            # 解包当前范围的最小和最大能量
            mask = (energies.value >= e_min) & (energies.value <= e_max)
            # 创建一个掩码以选择当前范围内的能量
            for flavor in Flavor:
                # 遍历每种中微子味道
                olum_nmo[energy_range][flavor][i] = np.trapezoid(ospec_nmo[flavor][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
                
                olum_imo[energy_range][flavor][i] = np.trapezoid(ospec_imo[flavor][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
            
                olum_nmo_tof[energy_range][flavor][i] = np.trapezoid(ospec_nmo_tof[flavor][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
              
                olum_imo_tof[energy_range][flavor][i] = np.trapezoid(ospec_imo_tof[flavor][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
               
    
    titles = ['Transformed (nmo_TOF)','Transformed (imo_TOF)']
    fig, axes = plt.subplots(2, 3, figsize=(20, 12), tight_layout=True)
    # 创建一个包含3x3子图的图形，并设置图形大小和布局
    
    smax = [0., 0., 0.]
    # 初始化列表以存储每个时期的最大通量值
    titles = [ 'Transformed (NMO)', 'TOF_NMO)']
    # 定义图表标题
    for idx, spec in enumerate([olum_nmo,  olum_nmo_tof]):
        # 遍历每种光谱类型（未转换、NMO、IMO）
        for jdx, phase in enumerate([burst_epoch, accretion_epoch, cooling_epoch]):
            # 遍历每个时期（爆发期、吸积期、冷却期）
            ax = axes[idx, jdx]
            # 获取当前子图轴
            timeunits = 'ms' if jdx == 0 else 's'
            # 将时间单位设置为毫秒（对于爆发期）或秒（对于其他时期）
                
            for energy_range in energy_ranges:
                # 遍历每个能量范围
                e_min, e_max = energy_range
                # 解包当前范围的最小和最大能量
                for flavor in Flavor:
                    # 遍历每种中微子味道
                    if idx == 0:
                        # 对于未转换光谱
                        smax[jdx] = np.maximum(smax[jdx], 1.1 * np.max(spec[energy_range][flavor][phase]))
                        # 更新当前时期的最大通量值
                        
                    ax.plot(times[phase].to(timeunits),
                            spec[energy_range][flavor][phase], label=f"{flavor.to_tex()} ({e_min}-{e_max} MeV)", lw=3,
                            color='C0' if flavor.is_electron else 'C1',
                            ls='-' if flavor.is_neutrino else ':')
                    # 绘制当前味道和能量范围的通量图
            
            ax.set(xlim=(times[phase][0].to(timeunits).value, times[phase][-1].to(timeunits).value),
                   ylim=(0, smax[jdx]))
            # 设置当前子图的x轴和y轴限制
            
            if jdx == 0:
                ax.set(ylabel=r'flux [$10^{16}$ erg$^{-1}$ cm$^{-2}$ s$^{-1}$]')
                ax.legend(loc='upper right', ncol=1, fontsize=18)
                # 设置y轴标签，并在右上角添加图例
            if jdx == 1:
                ax.set(title=titles[idx])
                # 为中间列设置标题
            if idx < 2:
                ax.set(xticklabels=[])
                # 对前两行移除x轴刻度标签
            else:
                ax.set(xlabel='time [{}]'.format(timeunits))
                # 为最后一行设置x轴标签
            
            ax.grid(ls=':')
            # 在当前子图中添加网格线

    return fig

In [ ]:
fig = plot_total_flux(model, AdiabaticMSW(), AdiabaticMSW(mh=MassHierarchy.INVERTED) )

In [ ]:
def plot_total_flux1(model, xform_nmo, xform_imo):
    """Plot initial and oscillated neutrino luminosities.
    
    Parameters
    ----------
    model : SupernovaModel
        An input model from a CCSN simulation.
    flav_xform : FlavorTransformation
        A FlavorTransformation subclass; used to create an instance.
    """
    
    energies = np.linspace(1,50,100) * u.MeV   
    D=10* u.kpc
    d = (10*u.kpc).to('cm').value # distance to SN
    mass=0.5* u.eV    
    times = model.get_time()
    burst_epoch = times <= 0.1*u.s
    accretion_epoch = (times > 0.1*u.s) & (times <= 0.5*u.s)
    cooling_epoch = (times > 0.5*u.s) & (times <= 10*u.s)
    energy_ranges = [(1, 10), (10, 30), (30, 50)]  # MeV
    
    
    olum_nmo = {E: np.zeros(len(times)) for E in energy_ranges}
    #olum_imo = {E: np.zeros(len(times)) for E in energy_ranges}
    olum_nmo_tof = {E: np.zeros(len(times)) for E in energy_ranges}
    #olum_imo_tof = {E: np.zeros(len(times)) for E in energy_ranges}

    # Compute the transformed and untransformed flux at each time.
    for i, t in enumerate(times):
        #ispec = model.get_initial_spectra(t, energies)
        ospec_nmo = model.get_transformed_spectra(t, energies, xform_nmo)
        #ospec_imo = model.get_transformed_spectra(t, energies, xform_imo)
        ospec_nmo_tof = model.get_delayed_flux(D, t, energies, mass, xform_nmo)
        #ospec_imo_tof,tof1 = model.get_delayed_flux(D, t, energies, mass, xform_imo)
        for energy_range in energy_ranges:
            # 遍历每个能量范围
            e_min, e_max = energy_range
            # 解包当前范围的最小和最大能量
            mask = (energies.value >= e_min) & (energies.value <= e_max)
            # 创建一个掩码以选择当前范围内的能量
            gc.collect()  # 强制垃圾回收
                # 遍历每种中微子味道
            olum_nmo[energy_range][i] = np.trapezoid(ospec_nmo[0][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
                
            #olum_imo[energy_range][i] = np.trapezoid(ospec_imo[0][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
            
            olum_nmo_tof[energy_range][i] = np.trapezoid(ospec_nmo_tof[0][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
              
            #olum_imo_tof[energy_range][i] = np.trapezoid(ospec_imo_tof[0][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
               
    
    titles = ['Transformed (nmo_TOF)','Transformed (imo_TOF)']
    fig, axes = plt.subplots(2, 3, figsize=(20, 12), tight_layout=True)
    # 创建一个包含3x3子图的图形，并设置图形大小和布局
    
    smax = [0., 0., 0.]
    # 初始化列表以存储每个时期的最大通量值
    # 定义图表标题
    for idx, (spec_olum, spec_ospec) in enumerate([(olum_nmo,  olum_nmo_tof)]):
        # 遍历每种光谱类型（未转换、NMO、IMO）
        for jdx, phase in enumerate([burst_epoch, accretion_epoch, cooling_epoch]):
            # 遍历每个时期（爆发期、吸积期、冷却期）
            ax = axes[idx, jdx]
            # 获取当前子图轴
            timeunits = 'ms' if jdx == 0 else 's'
            # 将时间单位设置为毫秒（对于爆发期）或秒（对于其他时期）
                
            for energy_range in energy_ranges:
                # 遍历每个能量范围
                e_min, e_max = energy_range
                # 解包当前范围的最小和最大能量
               
                if idx == 0:
                        # 对于未转换光谱
                    smax[jdx] = np.maximum(smax[jdx], 1.1 * np.max(spec_olum[energy_range][phase]))
                        # 更新当前时期的最大通量值
                        
                ax.plot(times[phase].to(timeunits),
                        spec_olum[energy_range][phase], label=f"{"NU_E"} ({e_min}-{e_max} MeV)", lw=3,
                        color='C0' ,
                        ls='-' )
                
                ax.plot(times[phase].to(timeunits),
                        spec_ospec[energy_range][phase], label=f"TOF {"NU_E"} ({e_min}-{e_max} MeV)", lw=3,
                        color= 'C1',
                        ls= ':')
                    # 绘制当前味道和能量范围的通量图
            
            ax.set(xlim=(times[phase][0].to(timeunits).value, times[phase][-1].to(timeunits).value),
                   ylim=(0, smax[jdx]))
            # 设置当前子图的x轴和y轴限制
            
            if jdx == 0:
                ax.set(ylabel=r'flux [$10^{16}$ erg$^{-1}$ cm$^{-2}$ s$^{-1}$]')
                ax.legend(loc='upper right', ncol=1, fontsize=18)
                # 设置y轴标签，并在右上角添加图例
            if jdx == 1:
                ax.set(title=titles[idx])
                # 为中间列设置标题
            if idx < 2:
                ax.set(xticklabels=[])
                # 对前两行移除x轴刻度标签
            else:
                ax.set(xlabel='time [{}]'.format(timeunits))
                # 为最后一行设置x轴标签
            
            ax.grid(ls=':')
            # 在当前子图中添加网格线

    return fig

fig = plot_total_flux1(model, AdiabaticMSW(), AdiabaticMSW(mh=MassHierarchy.INVERTED) )